<div style="text-align:center">Marco Seiz, Andreas Prahs, Anastasia August<br>
KIT, Institut für Angewandte Materialien (IAM)<br>
Mikrostrukturmodellierung und Simulation (MMS)
</div>
<h1>Mikrostruktursimulation <br/>Rechnerübung Berechnung von Phasendiagrammen WS 22/23</h1>

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib as mpl
import math
import numpy as np
from sympy import *
import time
from scipy.optimize import fsolve
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
init_printing()
mpl.rcParams.update({'font.size': 16})
mpl.rcParams['figure.figsize'] = (9,6)
mpl.rcParams['axes.grid'] = True
import warnings
from matplotlib import MatplotlibDeprecationWarning
warnings.filterwarnings('ignore', category=MatplotlibDeprecationWarning)


In [ ]:
R_val = 8.314 # universelle Gaskonstante
T_M_A_alpha_val = .8
T_M_B_alpha_val = 1.25
T_M_A_beta_val = 1.2
T_M_B_beta_val = .75
L_A_alpha_val = 60
L_B_alpha_val = 60
L_A_beta_val = 80
L_B_beta_val = 25
T_low_val = .6
T_high_val = 1.4

R = symbols('R', real=True)
T_M_A_alpha = symbols('T_{MA\\alpha}', real=True, positive=True)
T_M_B_alpha = symbols('T_{MB\\alpha}', real=True, positive=True)
T_M_A_beta = symbols('T_{MA\\beta}', real=True, positive=True)
T_M_B_beta = symbols('T_{MB\\beta}', real=True, positive=True)
L_A_alpha = symbols('L_{A\\alpha}', real=True, positive=True)
L_B_alpha = symbols('L_{B\\alpha}', real=True, positive=True)
L_A_beta = symbols('L_{A\\beta}', real=True, positive=True)
L_B_beta = symbols('L_{B\\beta}', real=True, positive=True)
T_low = symbols('T_{\\text{low}}', real=True, positive=True)
T_high = symbols('T_{\\text{high}}', real=True, positive=True)
x = symbols('x', real=True, positive=True)
T = symbols('T', real=True, positive=True)
xlab = "$x_A$ bzw. $1-x_B$"

def subs_constants(expr):
    return (expr.subs(R, R_val)
                .subs(T_M_A_alpha,T_M_A_alpha_val)
                .subs(T_M_B_alpha,T_M_B_alpha_val)
                .subs(T_M_A_beta, T_M_A_beta_val)
                .subs(T_M_B_beta, T_M_B_beta_val)
                .subs(L_A_alpha, L_A_alpha_val)
                .subs(L_B_alpha, L_B_alpha_val)
                .subs(L_A_beta, L_A_beta_val)
                .subs(L_B_beta, L_B_beta_val)
                .subs(T_low, T_low_val)
                .subs(T_high, T_high_val))

<h3>Aufgabe 1a)<br/>
Gibbsenergiegleichungen
</h3>

Es liegt ein Gemisch aus den Komponenten A und B vor, bei dem die drei Phasen $L$, $\alpha$, und $\beta$  beobachtet werden.
Weiterhin sind die Gibbsenergiekurven der drei Phasen als Funktion des Stoffmengenanteils $x_A = x$ und der Temperatur $T$ bekannt. Visualisieren Sie die Gibbsenergien für verschiedene Temperaturen und stellen Sie eine Hypothese auf, wie das Phasendiagramm aussehen wird.



In [ ]:
# Gibbs-Energie der flüssigen Phase
G_L = R*T*(x*ln(x)+(1-x)*ln(1-x))
G_L_f = lambdify([x, T], subs_constants(G_L))
display(G_L)

In [ ]:
# Gibbs-Energie der Phasen alpha, beta
G_alpha = R*T*(x*ln(x)+(1-x)*ln(1-x))-x*L_A_alpha*(T_M_A_alpha-T)/T_M_A_alpha-(1-x)*L_B_alpha*(T_M_B_alpha-T)/T_M_B_alpha
G_alpha_f = lambdify([x, T], subs_constants(G_alpha))
display(G_alpha)

In [ ]:
G_beta = R*T*(x*ln(x)+(1-x)*ln(1-x))-x*L_A_beta*(T_M_A_beta-T)/T_M_A_beta-(1-x)*L_B_beta*(T_M_B_beta-T)/T_M_B_beta
G_beta_f = lambdify([x, T], subs_constants(G_beta))
display(G_beta)

In [ ]:
xx = np.arange(.001,.999,.005)
Galpha = [subs_constants(G_alpha).subs(T,1.2).subs(x,xi) for xi in xx]
Gbeta = [subs_constants(G_beta).subs(T,1.2).subs(x,xi) for xi in xx]
GL = [subs_constants(G_L).subs(T,1.2).subs(x,xi) for xi in xx]
fig_Diff, ax_Diff = plt.subplots()
lineGa, = ax_Diff.plot(xx, Galpha, label=r"$G_\alpha$")
lineGb, = ax_Diff.plot(xx, Gbeta, label=r"$G_\beta$")
lineGL, = ax_Diff.plot(xx, GL, label="$G_L$")
ax_Diff.set_xlabel(xlab)
ax_Diff.set_ylabel("$G$")
ax_Diff.legend()
fig_Diff.canvas.draw()

def gibbsplot(T_val):
    Galpha = [G_alpha_f(xi, T_val) for xi in xx]
    Gbeta = [G_beta_f(xi, T_val)  for xi in xx]
    GL = [G_L_f(xi, T_val)  for xi in xx]
    lineGa.set_ydata(Galpha)
    lineGb.set_ydata(Gbeta)
    lineGL.set_ydata(GL)
    ymax = max(np.max(Galpha), np.max(Gbeta), np.max(GL))*1.05
    ymin = min(np.min(Galpha), np.min(Gbeta), np.min(GL))*1.05
    ax_Diff.set_ylim([ymin, ymax])
    ax_Diff.legend()
    fig_Diff.canvas.draw()

In [ ]:
interact(gibbsplot, T_val=widgets.BoundedFloatText(min=T_low_val, max=T_high_val, step=.05, value=1.2))

<h3>Aufgabe 1b)<br/>
Gleichgewichtsberechnung für ein Dreiphasengleichgewicht
</h3>

Überprüfen Sie, ob für das System eine eutektische Reaktion  vorliegt.  Diese ist in binären Systemen dadurch definiert, dass bei der eutektischen Temperatur die Flüssigphase mit beiden Festphasen im Gleichgewicht vorliegt und darunter in diese beiden Phasen zerfällt, oder in kurz: $L \to \alpha + \beta$

Gehen Sie hierzu wie folgt vor: Definieren Sie zunächst die vier nötigen Gleichgewichtsbedingungen zwischen den drei Phasen. Lösen Sie diese danach mit dem nsolve-Befehl. Schätzen Sie die Startwerte (initialvalues) für die eutektischte Temperatur und den Konzentrationen in den einzelnen Phasen mit Hilfe des Gibbsenergiediagramms ab.


Hinweis: Alle verwendeten Konstanten können in einen Ausdruck mit Hilfe von subs_constants(Ausdruck) eingesetzt werden.

In [ ]:
# Chemisches Potential der Phasen. Ableitungen der Gibbs Energien nach der Konzentration
dG_L_dx = diff(G_L,x)
display(dG_L_dx)
dG_alpha_dx = diff(G_alpha,x)
display(dG_alpha_dx)
dG_beta_dx = diff(G_beta,x)
display(dG_beta_dx)

In [ ]:
# Aufstellen der Gleichgewichte bei der eutektische Temperatur T_E
x_L = symbols('x_L', real=True, positive=True)
x_alpha = symbols(r'x_\alpha', real=True, positive=True)
x_beta = symbols(r'x_\beta', real=True, positive=True)
T_E = symbols('T_E', real=True, positive=True)
#die Reihenfolge der Gleichungen spielt keine Rolle, es müssen nur zwei Doppeltangenten vorliegen 
Gleichung1 = 
display(Gleichung1)
Gleichung2 = 
display(Gleichung2)
Gleichung3 = 
display(Gleichung3)
Gleichung4 = 
display(Gleichung4)


In [ ]:
eqs = [Gleichung1, Gleichung2, Gleichung3, Gleichung4 ]
subs_eqs = [subs_constants(x) for x in eqs]
unknowns = [T_E, x_L, x_alpha, x_beta] # die Reihenfolge hier gibt die Reihenfolge für die Startwerte vor!
# also: zuerst eutektische Temperatur, dann jeweils die Konzentration in der Schmelze, alpha und beta
# initivalues ist ebenso eine Liste, also [schätzung für T_e, schätzung für x_L, ...]
initialvalues = 



In [ ]:
subs_eqs # hier sollten nur noch die Unbekannten aus unknowns vorkommen

In [ ]:
sol = nsolve(subs_eqs,unknowns,initialvalues)
display(sol)

In [ ]:
Te = sol[0]
xle = sol[1]
xae = sol[2]
xbe = sol[3]
Te_val = float(Te)

<h3>Aufgabe 1c)<br/>
Tangentengleichung
</h3>

Stellen Sie nun die Tangente, die aus dem obigen Gleichgewicht folgt, graphisch dar. 
Nutzen Sie hierzu eine Gibbsenergiefunktion Ihrer Wahl sowie deren Ableitung und setzen Sie die Informationen des Dreiphasengleichgewichts via .subs() ein.

Welche Bedeutung hat der Schnittpunkt der Tangente mit den schwarzen Linien bei x=0 und x=1? Warum ist es unerheblich welche Phase zur Berechnung der Tangente genutzt wird?



In [ ]:
Tangente = 
display(Tangente) 

In [ ]:
fig_GTE, ax_GTE = plt.subplots()
ax_GTE.set_xlabel(xlab)
ax_GTE.set_ylabel("G")

xx = np.arange(.001,.999,.005)
G_alpha_plot = [subs_constants(G_alpha).subs(T,Te).subs(x,xi) for xi in xx]
ax_GTE.plot(xx,G_alpha_plot, label='$G_\\alpha$')
G_beta_plot = [subs_constants(G_beta).subs(T,Te).subs(x,xi) for xi in xx]
ax_GTE.plot(xx,G_beta_plot, label='$G_\\beta$')
G_L_plot = [subs_constants(G_L).subs(T,Te).subs(x,xi) for xi in xx]
ax_GTE.plot(xx,G_L_plot, label='$G_L$')
Tangente_plot = [Tangente.subs(x,xi) for xi in xx]
ax_GTE.plot(xx,Tangente_plot, label='Tangente')
ax_GTE.plot(xae, subs_constants(G_alpha).subs(x,xae).subs(T,Te),'x', label='$x_\\alpha$  bei $T_E$')
ax_GTE.plot(xbe, subs_constants(G_beta).subs(x,xbe).subs(T,Te),'x', label='$x_\\beta$ bei $T_E$')
ax_GTE.plot(xle, subs_constants(G_L).subs(x,xle).subs(T,Te),'x', label='$x_L$ bei $T_E$')
ax_GTE.axvline(0, color='black')
ax_GTE.axvline(1, color='black')
ax_GTE.legend(ncol=2)

<h3>Aufgabe 1d)<br/>
Gleichgewichtsberechnung für die Fest-Flüssig Phasenpaare
</h3>

Es soll nun das komplette Phasendiagramm berechnet werden. Berechnen Sie zunächst die Gleichgewichtspunkte aller Phasenpaare, in denen die flüssige Phase existiert. Beschriften Sie auch die Phasenbereiche in der Darstellung der einzelnen Phasenpaare.

Die Funktion calc_eq berechnet die Gleichgewichtskonzentrationen bei der gegebenen Temperatur für die Gibbsenergiefunktionen der Phasen.

In [ ]:
Num_Temp=25

In [ ]:
def calc_eq(Tval, G1expr, G2expr, mu1expr, mu2expr, x1sym, x2sym, x1ini, x2ini):
    Rsol = []
    if x1ini not in [0, 1.0] and x2ini not in [0, 1.0]:
        gleichung1 = subs_constants(simplify((mu1expr.subs(x,x1sym) - mu2expr.subs(x,x2sym)).subs(T,Tval)))
        gleichung2 = subs_constants(simplify(((G1expr+mu1expr*(x2sym-x1sym)).subs(x,x1sym) - G2expr.subs(x,x2sym))).subs(T,Tval))
        sol = nsolve([gleichung1, gleichung2], [x1sym, x2sym], [x1ini, x2ini])

        for solx in sol:
            if im(solx) != 0:
                Rsol.append(min(max(re(solx), 0), 1.0))
            else: Rsol.append(solx)
    else:
        Rsol.append(x1ini)
        Rsol.append(x2ini)
    return Rsol[0], Rsol[1]

In [ ]:
# Beispielaufruf: Bei der Temperatur 1.24 sollen die Gleichgewichte xla (Liquidus) und xal (Solidus)
# zwischen der Schmelze L und der Festphase alpha berechnet werden. Da 1.24 nah am Schmelzpunkt von reinem A liegt,
# werden die Startwerte für das numerische Verfahren am linken Rand des Phasendiagramms gewählt.
xla = symbols(r'x_{L\alpha}',real=True, positive=True)
xal = symbols(r'x_{\alpha\ L}', real=True, positive=True)
calc_eq(1.24, G_L, G_alpha, dG_L_dx, dG_alpha_dx, xla, xal, 0.2, 0.01)

In [ ]:
# Gleichgewicht alpha-liquid
xla = symbols('x_{L\\alpha}',real=True, positive=True)
xal = symbols('x_{\\alpha\ L}', real=True, positive=True)

#x_aL_val, x_L1_val = calc_xaL_xL1(1.2)
#print("x_aL=",x_aL_val," x_L1=",x_L1_val)
Solidus_aL=[]
Solidus_aT=[]
Liquidus_La=[]
Liquidus_Ta=[]

xla_val = 0.9
xal_val = 0.8
Tlow = min(T_M_A_alpha_val, T_M_B_alpha_val)
Thigh =  max(T_M_A_alpha_val, T_M_B_alpha_val)
for Ti in range(1,Num_Temp):
    T_val = Tlow-(Tlow-Thigh)*Ti/(Num_Temp+1)
    xla_val, xal_val = calc_eq(
        T_val, G_L, G_alpha, dG_L_dx, dG_alpha_dx,
        xla, xal, xla_val, xal_val) 
    print("T=%f"%T_val,"x_aL=",xal_val," x_L1=",xla_val)
    Solidus_aL.append(xal_val)
    Solidus_aT.append(T_val)
    Liquidus_La.append(xla_val)
    Liquidus_Ta.append(T_val)
    
Solidus_aL=np.array(Solidus_aL)
Solidus_aT=np.array(Solidus_aT)
Liquidus_La=np.array(Liquidus_La)
Liquidus_Ta=np.array(Liquidus_Ta)
    


In [ ]:
fig_SL1, ax_SL1 = plt.subplots()
ax_SL1.set_xlabel(xlab)
ax_SL1.set_ylabel("T")
ax_SL1.plot(Solidus_aL, Solidus_aT, '-xb', label=r'Solidus $\alpha$L')
ax_SL1.plot(Liquidus_La, Liquidus_Ta, '-xr', label=r'Liquidus L$\alpha$')
ax_SL1.legend()
#ax_SL1.text(0.5, 0.95 ,r"Beispiel $\gamma^2$",fontsize=25) # x, y wie im Koordinatensystem, gefolgt vom Text


In [ ]:
xbl = symbols(r'x_{\beta\ L}',real=True, positive=True)
xlb = symbols(r'x_{L\beta}', real=True, positive=True)
Solidus_bL=[]
Solidus_bT=[]
Liquidus_Lb=[]
Liquidus_Tb=[]

xlb_val = 0.01
xbl_val = 0.1
Tlow = min(T_M_A_beta_val, T_M_B_beta_val) 
Thigh =  max(T_M_A_beta_val, T_M_B_beta_val) 
for Ti in range(1,Num_Temp):
    T_val = Tlow-(Tlow-Thigh)*Ti/(Num_Temp+1)
    xlb_val, xbl_val = calc_eq(
        T_val, G_L, G_beta, dG_L_dx, dG_beta_dx,
        xlb, xbl, xlb_val, xbl_val) 
    print("T=%f"%T_val,"x_bL=",xbl_val," x_Lb=",xlb_val)
    Solidus_bL.append(xbl_val)
    Solidus_bT.append(T_val)
    Liquidus_Lb.append(xlb_val)
    Liquidus_Tb.append(T_val)
    
    
Solidus_bL=np.array(Solidus_bL)
Solidus_bT=np.array(Solidus_bT)
Liquidus_Lb=np.array(Liquidus_Lb)
Liquidus_Tb=np.array(Liquidus_Tb)

In [ ]:
fig_SL2, ax_SL2 = plt.subplots()
ax_SL2.set_xlabel("$x_A$ bzw. 1-$x_B$")
ax_SL2.set_ylabel("T")
ax_SL2.plot(Solidus_bL, Solidus_bT, '-xk', label=r'Solidus $\beta$L')
ax_SL2.plot(Liquidus_Lb, Liquidus_Tb, '-xm', label=r'Liquidus $L\beta$')
ax_SL2.legend()
#ax_SL2.text(0.4, 1.06 ,r"Beispiel $\gamma^2$",fontsize=25) # x, y wie im Koordinatensystem, gefolgt vom Text


In [ ]:
fig_SL12, ax_SL12 = plt.subplots()
ax_SL12.set_xlabel(xlab)
ax_SL12.set_ylabel("T")
ax_SL12.plot(Solidus_aL, Solidus_aT, '-xb', label=r'Solidus $\alpha$L')
ax_SL12.plot(Liquidus_La, Liquidus_Ta, '-xr', label=r'Liquidus $L\alpha$')
ax_SL12.plot(Solidus_bL, Solidus_bT, '-ok', label=r'Solidus $\beta$L')
ax_SL12.plot(Liquidus_Lb, Liquidus_Tb, '-om', label=r'Liquidus $L\beta$')
ax_SL12.axvline(0, color='black')
ax_SL12.axvline(1, color='black')
ax_SL12.legend(ncol=2)

Was impliziert der Schnittpunkt der Liquiduskurven? Warum kann dieselbe Argumentation nicht für den Schnittpunkt der Soliduskurven angewandt werden?




<h3>Aufgabe 1e)<br/>
Gleichgewichtsberechnung für die Fest-Fest Phasenpaare
</h3>

In [ ]:
xab = symbols(r'x_{\alpha\ \beta}',real=True, positive=True)
xba = symbols(r'x_{\beta\ \alpha}', real=True, positive=True)
    
Solvus_ab=[]
Solvus_ba=[]
Solvus_abT=[]
Solvus_baT=[]
Num_Temp2 = Num_Temp
Num_Temp = 15

# A ist links, B ist rechts
xab_val = 0.1
xba_val = 0.9
Thigh = Te + 0.05
Tlow = Te-0.05
for Ti in range(1,Num_Temp):
    T_val = Tlow-(Tlow-Thigh)*Ti/(Num_Temp+1)
    xab_val, xba_val = calc_eq(
        T_val, G_alpha, G_beta, dG_alpha_dx, dG_beta_dx,
        xab, xba, xab_val, xba_val) 
    print("T=%f"%T_val,"x_ab=",xab_val," x_ba=",xba_val)
    Solvus_ab.append(xab_val)
    Solvus_abT.append(T_val)
    Solvus_ba.append(xba_val)
    Solvus_baT.append(T_val)
    
Solvus_ab=np.array(Solvus_ab)
Solvus_ba=np.array(Solvus_ba)
Solvus_abT=np.array(Solvus_abT)
Solvus_baT=np.array(Solvus_baT)
Num_Temp = Num_Temp2


In [ ]:
fig_Sol, ax_Sol = plt.subplots()
ax_Sol.set_xlabel("$x_A$ bzw. 1-$x_B$")
ax_Sol.set_ylabel("T")
ax_Sol.plot(Solvus_ab, Solvus_abT, 'vc', label=r'Solvus $\alpha \to \alpha+\beta$')
ax_Sol.plot(Solvus_ba, Solvus_baT, '^c', label=r'Solvus $\beta \to \alpha+\beta$')
ax_Sol.legend()

In [ ]:
fig_all, ax_all = plt.subplots(figsize=(10,9))
ax_all.set_xlabel(xlab)
ax_all.set_ylabel("T")
ax_all.plot(Solidus_aL, Solidus_aT, '-xb', label=r'Solidus $\alpha$L')
ax_all.plot(Liquidus_La, Liquidus_Ta, '-xr', label=r'Liquidus $L\alpha$')
ax_all.plot(Solidus_bL, Solidus_bT, '-ok', label=r'Solidus $\beta$L')
ax_all.plot(Liquidus_Lb, Liquidus_Tb, '-om', label=r'Liquidus $L\beta$')
ax_all.plot(Solvus_ab, Solvus_abT, '-vc', label=r'Solvus $\alpha \to \alpha+\beta$')
ax_all.plot(Solvus_ba, Solvus_baT, '-^c', label=r'Solvus $\beta \to \alpha+\beta$')
ax_all.legend(loc='lower left', bbox_to_anchor= (0.0, 1.01), ncol=3)
ax_all.axvline(0, color='black')
ax_all.axvline(1, color='black')

<h3>Aufgabe 1f)<br/>
Stabiles Phasendiagramm
</h3>

Bestimmen Sie nun aufgrund der vorhergehenden Aufgaben das stabile Phasendiagramm und stellen Sie dieses dar.

Sie können die Ergebnisse filtern, indem sie eine Bedingung als Index verwenden. Als Bedingung können sie direkt einen Vergleich zwischen einem Array und einer Zahl benutzen, die Zahl wird dann gegen jedes Element des Arrays verglichen. Im folgenden Beispiel werden nur die Elemente von Solidus_bL ausgegeben, bei denen an der selben Stelle die Solidustemperatur Solidus_bT größer oder gleich 1.1 ist.

Hinweis: Die eutektische Temperatur ist in der Variablen Te_val verfügbar.

In [ ]:
Solidus_bL[Solidus_bT >= 1.1]

In [ ]:
Solidus_stab_bL = Solidus_bL[
Solidus_stab_bT = Solidus_bT[
Solidus_stab_aL = Solidus_aL[
Solidus_stab_aT = Solidus_aT[

Liquidus_stab_Lb = Liquidus_Lb[
Liquidus_stab_Tb = Liquidus_Tb[
Liquidus_stab_La = Liquidus_La[
Liquidus_stab_Ta = Liquidus_Ta[

Solvus_stab_ab = Solvus_ab[
Solvus_stab_ba = Solvus_ba[
Solvus_stab_abT = Solvus_abT[
Solvus_stab_baT = Solvus_baT[



In [ ]:
fig_all, ax_all = plt.subplots(figsize=(10,10))
ax_all.set_xlabel(xlab)
ax_all.set_ylabel("T")
ax_all.plot(Solidus_stab_aL, Solidus_stab_aT, '-ob',  label=r'Solidus $\alpha$L')
ax_all.plot(Liquidus_stab_La, Liquidus_stab_Ta, '-or', label=r'Liquidus $L\alpha$')
ax_all.plot(Solidus_stab_bL, Solidus_stab_bT, '-xk', label=r'Solidus $\beta$L')
ax_all.plot(Liquidus_stab_Lb, Liquidus_stab_Tb, '-xm', label=r'Liquidus $L\beta$')

ax_all.plot(Solvus_stab_ab, Solvus_stab_abT, '-vc', label=r'Solvus $\alpha \to \alpha+\beta$')
ax_all.plot(Solvus_stab_ba, Solvus_stab_baT, '-^c', label=r'Solvus $\beta \to \alpha+\beta$')
ax_all.hlines(float(Te), float(xae), float(xbe), label="Eutektikale", color="black") 
ax_all.legend(loc='lower left', bbox_to_anchor= (0.0, 1.01), ncol=3)
fig_all.tight_layout()
ax_all.axvline(0, color='black')
ax_all.axvline(1, color='black')
fig_all.savefig("phasendiagramm.pdf")

Das Phasendiagramm ist im Moment etwas ungünstig und grob gerastert.
Erhöhen Sie die Variable Num_Temp bei 1d) auf 50 und ändern Sie die Variable Tlow in den Zellen von 1d) auf Te-0.01.
Berechnen Sie danach das Phasendiagramm erneut, indem Sie im Reiter oben auf "Kernel" und dann "Restart & Run All" klicken. 

Während der Berechnung können Sie mit der nächsten Aufgabe fortfahren, das vorläufige Phasendiagramm liegt im Ordner als "phasendiagramm.pdf" gespeichert vor.

<h3>Aufgabe 1g)<br/>
Mikrostrukturentwicklung
</h3>

Wie sieht die Mikrostruktur einer Legierung der Zusammensetzung $x_A=0.4$ aus, wenn langsam aus der Schmelze bis $T=1.1$ abgekühlt wird? Was passiert mit einer Legierung von eutektischer Zusammensetzung? Gehen Sie dabei auf die Morphologie sowie die Massenanteile der Phasen und des Eutektikums ein.




Was passiert, wenn man diese Mikrostrukturen wieder über die Liquidustemperatur aufheizt?


